# First NGS-Py example
We are solving the Poisson equation 

$$\text{find: } u \in H_{0,D}^1 \quad \int_\Omega \nabla u \nabla v = \int_\Omega f v \quad \forall v \in H_{0,D}^1$$

import NGSolve and Netgen Python modules:

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

Generate an unstructured mesh of maximal mesh-size 0.2:

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.nv, mesh.ne

Declare a finite element space:

In [3]:
fes = H1(mesh, order=2, dirichlet="bottom|right")
fes.ndof

125

Declare test and trial functions

In [4]:
u = fes.TrialFunction()
v = fes.TestFunction()

define and assemble linear and bilinear forms:

In [5]:
a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI (grad(u)*grad(v))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(x*v)
f.Assemble()

# print (f.vec)
# print (a.mat)

GridFunction: a finite element function, contains coefficient vector

In [6]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * f.vec

# print (fes.FreeDofs())
# print (gfu.vec)

In [7]:
help (fes)

Help on FESpace in module ngsolve.comp object:

class FESpace(builtins.object)
 |  Finite Element Space
 |  
 |  Provides the functionality for finite element calculations. Use
 |  the finite element space generator functions to construct the space,
 |  this can sometimes provide additional functionality (e.g HCurl).
 |  When createing a FESpace with a generator function the set parameters
 |  are passed to the FESpace constructor with and the type parameter is
 |  set. If the space has additional functionality it is added to the space.
 |  
 |  Available generator functions
 |  
 |  H1
 |  HCurl
 |  HDiv
 |  L2
 |  FacetFESpace
 |  HDivDiv
 |  
 |  2 __init__ overloads:
 |    1) To create a registered FESpace
 |    2) To create a compound FESpace from multiple created FESpaces
 |  
 |  1)
 |  
 |  Parameters
 |  
 |  type : string
 |    Type of the finite element space. This parameter is automatically
 |    set if the space is constructed with a generator function.
 |  
 |  mesh : ngs

In [8]:
print (x*v)

coef binary operation '*', real, dim=1
  coef coordinate x, real, dim=1
  coef N5ngfem13ProxyFunctionE, real, dim=1



In [9]:
Draw (gfu)